In [1]:
!pip install openpyxl

In [2]:
!pip3 install torch torchvision torchaudio

In [3]:
!pip install torchvision transformers  

     |████████████████████████████████| 3.4 MB 4.6 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 61 kB 536 kB/s 
     |████████████████████████████████| 596 kB 53.2 MB/s 
     |████████████████████████████████| 895 kB 34.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# Importar librerías
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from seaborn import load_dataset

pd.options.mode.chained_assignment = None  # Evita error al copiar df. Default='warn'

In [5]:
categories = {
    '1 star': 'muy negativo',
    '2 stars': 'negativo',
    '3 stars': 'neutro',
    '4 stars': 'positivo',
    '5 stars': 'muy positivo'
}
class NlpTown:
    
    def __init__(self):
        # https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment 
        self.tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
        self.model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
        self.nlp = pipeline('sentiment-analysis', model=self.model, tokenizer=self.tokenizer)
    
    def analyse(self, text):
        try:
            output = self.nlp(text)
            output = output[0]['label']
            return categories[output]
        except:
            return 'error'

In [6]:
n = NlpTown()

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
#Importar archivo exel
File = pd.ExcelFile('04 Datos Limpios.xlsx')
print(File.sheet_names)

['Posts', 'Comments']


In [ ]:
df_posts=File.parse('Posts')

In [ ]:
df_comments=File.parse('Comments')

In [ ]:
df_posts

,created_date,created_time,message,classification,format,page_id,page_name,page_name_id,post_id,react_angry,react_haha,react_like,react_love,react_sad,react_wow,react_care,share,region,kind,political_party
0,2021-04-02,21:17:00,Amigos y amigas: Esta tarde registramos mi can...,Aviso,"Texto,Imágenes",105287118287766,Irene Herrera,irene.herreraaa,121727003310444,0,1,241,98,0,0,7,70,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
1,2021-04-06,12:47:00,Memo Topete es mi amigo de toda la vida. Un ho...,Aviso,"Texto,Imágenes",105287118287766,Irene Herrera,irene.herreraaa,122949746521503,0,1,123,26,0,0,3,27,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
2,2021-04-06,21:38:00,"Hoy arranqué campaña en mi barrio, El Túnel, e...",Aviso,"Texto,Imágenes",105287118287766,Irene Herrera,irene.herreraaa,123127233170421,0,2,154,70,0,0,4,69,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
3,2021-04-06,21:59:00,Acompáñame al recorrido por las calles del cen...,Aviso,"Texto,Imágenes",105287118287766,Irene Herrera,irene.herreraaa,123131033170041,0,3,161,45,0,0,3,47,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
4,2021-04-07,08:21:00,Te invito a que escuches la entrevista que me ...,Aviso,"Texto,Imágenes",105287118287766,Irene Herrera,irene.herreraaa,123285036487974,0,1,98,23,0,0,1,12,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,2021-05-30,11:26:00,¡Manzanillo nos llama! 💚💚💚 ¡Vamos juntos a dev...,Motivacional,"Texto,Video",105287118287766,Irene Herrera,irene.herreraaa,1186903875064640,0,6,274,74,0,0,6,60,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
153,2021-05-31,19:17:00,¡La Ola Verde ya arrasó en Manzanillo! 💚💚💚 Aco...,Aviso,"Texto,Infografía",105287118287766,Irene Herrera,irene.herreraaa,152238653592612,0,0,201,46,0,0,2,77,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
154,2021-06-01,12:21:00,¡Hoy es nuestro cierre de campaña! ✅✅✅ Nos vem...,Aviso,"Texto,Infografía",105287118287766,Irene Herrera,irene.herreraaa,152737313542746,0,3,174,45,0,0,2,69,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México
155,2021-06-01,23:42:00,¡Manzanillo ya ganó! 💚💚💚 Hoy al calor de miles...,Aviso,"Texto,Imágenes",105287118287766,Irene Herrera,irene.herreraaa,153071386842672,0,1,292,124,0,0,5,55,Manzanillo,Presidencia Municipal,Partido Verde Ecologista de México


In [ ]:
df_comments

,post_id,comment_id,profile_id,created_date,created_time,from_name,message,gender,reactions,page_name
0,121727003310444,1,100001460864870,2021-04-03,03:18:00,Marisela Baron Olivera,Muy bien Irene Herrera hay estaremos apoyándote😉,M,1,Irene Herrera
1,121727003310444,2,100044234033635,2021-04-03,03:20:00,Virgilio Mendoza,Vamos #ConTodo amiga ✅✅✅,H,22,Irene Herrera
2,121727003310444,3,100005208459189,2021-04-04,06:17:00,Adrian Mendoza Sandoval,Virgilio Mendoza por un mejor Colima ✅✅✅,H,0,Irene Herrera
3,121727003310444,4,100044349433235,2021-04-03,03:21:00,Gaby Benavides,Llegó la #OlaVerde 💚🌊,M,23,Irene Herrera
4,121727003310444,5,100005922013495,2021-04-03,03:37:00,Guillermo Rivera,Gaby Benavides con todo Amiga..!!,H,0,Irene Herrera
...,...,...,...,...,...,...,...,...,...,...
5512,236112457860954,5513,100011612551326,2021-06-03,13:26:00,Imelda Guzman,Vamos verde 💚💚💚💚💚💚💚💚💚💚💚💚💚💚,M,0,Irene Herrera
5513,236112457860954,5514,100039402063084,2021-06-03,14:22:00,Yerania Esquibel,Vamos verde✅✅🍀\n [PHOTO] https://scontent.fhkg...,M,0,Irene Herrera
5514,236112457860954,5515,100004522110874,2021-06-03,15:56:00,María Laura Macias Ramirez,💚💚💚💚,M,0,Irene Herrera
5515,236112457860954,5516,100006010063472,2021-06-03,17:13:00,Yessi Chávez Arreola,💚💚💚💚 vamos verdes 👏👏👏👏💪,M,0,Irene Herrera


In [ ]:
# Método para limpieza
def clean_text(text):
    text = str(text)
    text = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", '', text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # para menciones
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'[\.\,]', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'\[PHOTO\]','', text)
    text = " ".join(text.split())
    return text

In [ ]:
# Limpiar comentarios
df_comments['message'] = df_comments['message'].apply(clean_text)

In [ ]:
# Eliminar comentarios sin contenido
df_comments = df_comments[df_comments['message'] != ''].reset_index(drop=True)

In [ ]:
# Clasificar sentimientos de comentarios
df_comments['feeling'] = df_comments['message'].apply(n.analyse)

In [ ]:
# Delete rows with error
df_comments = df_comments[df_comments['feeling'] != 'error'].reset_index(drop=True)

In [ ]:
df_comments

,post_id,comment_id,profile_id,created_date,created_time,from_name,message,gender,reactions,page_name,feeling
0,121727003310444,1,100001460864870,2021-04-03,03:18:00,Marisela Baron Olivera,Muy bien Irene Herrera hay estaremos apoyándote😉,M,1,Irene Herrera,muy positivo
1,121727003310444,2,100044234033635,2021-04-03,03:20:00,Virgilio Mendoza,Vamos amiga ✅✅✅,H,22,Irene Herrera,muy positivo
2,121727003310444,3,100005208459189,2021-04-04,06:17:00,Adrian Mendoza Sandoval,Virgilio Mendoza por un mejor Colima ✅✅✅,H,0,Irene Herrera,muy positivo
3,121727003310444,4,100044349433235,2021-04-03,03:21:00,Gaby Benavides,Llegó la 💚🌊,M,23,Irene Herrera,muy negativo
4,121727003310444,5,100005922013495,2021-04-03,03:37:00,Guillermo Rivera,Gaby Benavides con todo Amiga !!,H,0,Irene Herrera,muy positivo
...,...,...,...,...,...,...,...,...,...,...,...
4577,236112457860954,5513,100011612551326,2021-06-03,13:26:00,Imelda Guzman,Vamos verde 💚💚💚💚💚💚💚💚💚💚💚💚💚💚,M,0,Irene Herrera,muy positivo
4578,236112457860954,5514,100039402063084,2021-06-03,14:22:00,Yerania Esquibel,Vamos verde✅✅🍀,M,0,Irene Herrera,muy positivo
4579,236112457860954,5515,100004522110874,2021-06-03,15:56:00,María Laura Macias Ramirez,💚💚💚💚,M,0,Irene Herrera,muy positivo
4580,236112457860954,5516,100006010063472,2021-06-03,17:13:00,Yessi Chávez Arreola,💚💚💚💚 vamos verdes 👏👏👏👏💪,M,0,Irene Herrera,muy positivo


In [ ]:
# Crear nuevo archivo xlsx con los datos limpios y clasificados
Posts = df_posts
Comments = df_comments
writer = pd.ExcelWriter('nombre_archivo.xlsx')
Posts.to_excel(writer, sheet_name="Posts", index=False)
Comments.to_excel(writer, sheet_name="Comments", index=False)
writer.save()
writer.close()